# Calculating Hotel Occupancy

---

> Hotel occupancy is a critical factor during the booking process and can provide additional insight into the likelihood of cancellations and/or forecasting future ADR.
> 
> However, *there's no clear indication of the total number of guest rooms for either hotel.*
>
> 
> I will determine the maximum number of rooms occupied for each date for each hotel, which can be used as a placeholder max occupancy number.

---

# Import Packages and Read Data

In [ ]:
## Used to upload 
%load_ext autoreload
%autoreload 2

In [ ]:
## Enabling access to custom functions in separate directory

# Import necessary modules
import os
import sys

# Construct the absolute path to the 'src' directory
src_path = os.path.abspath(os.path.join('../..', 'src'))

# Append the path to 'sys.path'
if src_path not in sys.path:
    sys.path.append(src_path)

import db_utils, eda

## Data Handling
import pandas as pd
import numpy as np

import datetime as dt

## Visualizations
import matplotlib.pyplot as plt
# import plotly.express as px
# import seaborn as sns
import sweetviz as sv

## Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 50)
%matplotlib inline

## Load Pre-Reviewed Data

In [ ]:
data_path = '../../data/2.2_temporally_updated_data.feather'
df_data = pd.read_feather(data_path)
df_data.head()

In [ ]:
for col in df_data.select_dtypes('number').columns:

    df_data[col] = pd.to_numeric(df_data[col], downcast = 'float')

In [ ]:
df_data.info()

# Calculate Daily Occupancies

# Hotel 1

In [ ]:
df_data['HotelNumber'].unique()

In [ ]:
hotel = 'H1'

hotel_filter = (df_data['HotelNumber'] == hotel)

df_data_h1 = df_data[hotel_filter]

df_data_h1

In [ ]:
min_date = df_data_h1['ArrivalDate'].min()
max_date = df_data_h1['DepartureDate'].max()

print(f'The earliest date is: {min_date}.\nThe latest date is: {max_date}.')

### ChatGPT-Generated Code

#### Walkthrough of Code Below

Certainly! Let's break down the corrected code into its key steps, explaining each part and its purpose in calculating the number of active hotel reservations for each date:

**3. Generate Counts for Arrivals and Departures**

```python
arrivals = df_data_h1['ArrivalDate'].value_counts().rename('count')
departures = df_data_h1['DepartureDate'].value_counts().rename('count')
```
- Counts how many reservations start (`arrivals`) and end (`departures`) on each date. The `value_counts()` method tallies occurrences of each date, and `rename('count')` changes the Series name to `'count'`, which aids in clarity for later operations.

**4. Combine Arrival and Departure Counts**

```python
df_counts = pd.concat([arrivals, -departures]).sort_index().reset_index()
df_counts.columns = ['Date', 'Count']
```
- Combines the arrivals and departures into a single DataFrame, `df_counts`, with arrivals contributing positively to the count and departures negatively (indicating the end of a reservation). The data is then sorted by date.

**5. Aggregate Counts on the Same Date**

```python
df_counts = df_counts.groupby('Date').sum()
```
- Since the combination of arrivals and departures could result in multiple entries for the same date, this step aggregates (sums) all counts for each date. This ensures each date is unique, addressing the initial issue of duplicate labels.

**6. Generate a Complete Date Range and Reindex**

```python
date_range = pd.date_range(start=df_counts.index.min(), end=df_counts.index.max())
df_counts = df_counts.reindex(date_range, fill_value=0)
```
- Creates a continuous range of dates covering the entire period from the earliest to the latest date in `df_counts`. It then reindexes `df_counts` to include every date in this range, filling any dates without data with `0`, ensuring there's a record for every single day in the period.

**7. Calculate Cumulative Sum for Active Reservations**

```python
df_counts['Active_Reservations'] = df_counts['Count'].cumsum()
```
- Computes the cumulative sum of the daily net reservation counts (`Count`). This step effectively calculates the total number of active reservations for each date by adding up the arrivals and subtracting the departures as they occur over time.

**Conclusion**

- The final output, `df_counts['Active_Reservations']`, shows the total number of active reservations for each date in the range. This method is efficient and avoids the problem of duplicate labels by ensuring that each date is unique before the reindexing step, leveraging pandas' capabilities for handling time series data.

### Code

In [ ]:
def get_counts(dataframe, arrivaldate, departuredate, name = 'count'):
    
    '''Generate counts for arrivals and departures on their respective dates.'''
    
    arrivals = (dataframe[arrivaldate]
                .value_counts()
                .rename(name))
    
    departures = (dataframe[departuredate]
                  .value_counts()
                  .rename(name))
    
    return arrivals, departures


def aggregate_counts_by_date(arrivals, departures):
    
    '''Create a DataFrame from arrivals and departures,
    marking departures as negative.'''
    
    df_counts = (pd.concat([arrivals, -departures])
                 .sort_index()
                 .reset_index())
    
    df_counts.columns = ['Date', 'Count']
    
    return df_counts.groupby('Date').sum()


def generate_date_range(df_counts):
    '''Generate a complete date range covering the period in df_data.'''
    
    return pd.date_range(start=df_counts.index.min(),
                         end=df_counts.index.max())


def reindex_and_fill_zero(df_counts, date_range):
    '''Reindex the aggregated count DataFrame to include all dates in the range,
    filling missing dates with 0'''
    
    return df_counts.reindex(date_range, fill_value=0)


def calculate_daily_active_res(df_counts):
    '''Calculate the cumulative sum to determine active reservations for each date'''
   
    return df_counts['Count'].cumsum()


def calculate_daily_occupancy(dataframe, arrivaldate, departuredate, name = 'count'):
    
    arrivals, departures = get_counts(dataframe, arrivaldate, departuredate, name = 'count')
    
    daily_counts = aggregate_counts_by_date(arrivals, departures)
    
    date_range = generate_date_range((daily_counts))
    
    df_reindexed = reindex_and_fill_zero(daily_counts, date_range)
    
    return calculate_daily_active_res(df_reindexed)

In [ ]:
df_counts = calculate_daily_occupancy(df_data_h1, 'ArrivalDate', 'DepartureDate')
df_counts.name = 'Active_Reservations'
df_counts

In [ ]:
# # Generate counts for arrivals and departures on their respective dates
# arrivals = df_data_h1['ArrivalDate'].value_counts().rename('count')
# departures = df_data_h1['DepartureDate'].value_counts().rename('count')
# arrivals.head(), departures.head()

In [ ]:
# # Create a DataFrame from arrivals and departures, marking departures as negative
# df_counts = pd.concat([arrivals, -departures]).sort_index().reset_index()
# df_counts.columns = ['Date', 'Count']
# df_counts

In [ ]:
# # Aggregate counts on the same date to avoid duplicate labels
# df_counts = df_counts.groupby('Date').sum()
# df_counts

In [ ]:
# # Generate a complete date range covering the period in df_data
# date_range = pd.date_range(start=df_counts.index.min(),
#                            end=df_counts.index.max())
# date_range

In [ ]:
# # Reindex the aggregated count DataFrame to include all dates in the range, filling missing dates with 0
# df_counts = df_counts.reindex(date_range, fill_value=0)
# df_counts

In [ ]:
# # Calculate the cumulative sum to determine active reservations for each date
# df_counts['Active_Reservations'] = df_counts['Count'].cumsum()

# df_counts['Active_Reservations']

#### Groupby.Sum vs. Cumsum

The use of both `groupby().sum()` and the `cumsum()` methods serves two different purposes in the process of calculating the total number of active reservations for each date. Here's a clarification of the roles each step plays in the computation:

**1. GroupBy().sum()**

- **Purpose:** This step aggregates the daily net changes in reservations (arrivals and departures) for each unique date. Since arrivals are counted positively and departures negatively, the sum for each date tells us the net reservation change on that day. 
- **What It Solves:** If, for instance, 5 reservations start (arrive) and 3 end (depart) on a particular date, the net change in reservations for that day would be +2. This calculation consolidates all changes into a single value per date, ensuring there's no duplication of dates in the dataset, which is necessary for the next steps.

**2. cumsum()**

- **Purpose:** The cumulative sum (`cumsum()`) takes these daily net changes and accumulates them over the entire period to calculate the total number of active reservations for each date. It essentially adds up the net changes from the start date, rolling forward, to show how many reservations are active on any given day.
- **What It Solves:** This step provides the running total of active reservations. It accounts for the ongoing balance of reservations as they begin and end over time, showing the total active reservations on each date. This is crucial for understanding the capacity or occupancy on any given day.

**Illustrative Example:**

Let's say you have data for three days:

- **Day 1:** 5 arrivals, 0 departures (net +5)
- **Day 2:** 3 arrivals, 1 departure (net +2)
- **Day 3:** 2 arrivals, 4 departures (net -2)

After `groupby().sum()`, you'd have a net change sequence of [+5, +2, -2].

Applying `cumsum()` to this sequence gives you the total active reservations for each day: [5, 7, 5]. This demonstrates how the occupancy evolves:

- **Day 1:** Starts with 5,
- **Day 2:** Increases to 7,
- **Day 3:** Decreases back to 5.

**Conclusion:**

- **`groupby().sum()`** is used for condensing the dataset into a form where each date has a single net change value, resolving any issues with duplicate dates.
- **`cumsum()`** transforms these net changes into a running total of active reservations, reflecting how the number of active reservations builds up or reduces over time.

## Adding Arrival/Departure Occupancies to Original Data

In [ ]:
df_counts.head(10)

In [ ]:
# Define active_reservations
active_reservations = df_counts

# Find the maximum occupancy
max_occupancy = active_reservations.max()

# Map the occupancy on arrival and departure dates to each reservation
df_data_h1.loc[:, 'occupancy_at_arrival'] = df_data_h1.loc[:, 'ArrivalDate'].map(active_reservations)
df_data_h1.loc[:, 'occupancy_at_departure'] = df_data_h1.loc[:, 'DepartureDate'].map(active_reservations)

# Convert these occupancies to percentages of the maximum occupancy
df_data_h1.loc[:, 'occupancy_pct_at_arrival'] = (df_data_h1.loc[:, 'occupancy_at_arrival'] / max_occupancy) * 100
df_data_h1.loc[:, 'occupancy_pct_at_departure'] = (df_data_h1.loc[:, 'occupancy_at_departure'] / max_occupancy) * 100

df_data_h1

In [ ]:
df_data_h1

# Hotel 2

In [ ]:
hotel = 'H2'

hotel_filter = (df_data['HotelNumber'] == hotel)

df_data_h2 = df_data[hotel_filter]

df_counts_h2 = calculate_daily_occupancy(df_data_h2, 'ArrivalDate', 'DepartureDate')
df_counts_h2.name = 'Active_Reservations'
df_counts_h2

In [ ]:
# Define active_reservations
active_reservations = df_counts

# Find the maximum occupancy
max_occupancy = active_reservations.max()

# Map the occupancy on arrival and departure dates to each reservation
df_data_h2.loc[:, 'occupancy_at_arrival'] = df_data_h2.loc[:, 'ArrivalDate'].map(active_reservations)
df_data_h2.loc[:, 'occupancy_at_departure'] = df_data_h2.loc[:, 'DepartureDate'].map(active_reservations)

# Convert these occupancies to percentages of the maximum occupancy
df_data_h2.loc[:, 'occupancy_pct_at_arrival'] = (df_data_h2.loc[:, 'occupancy_at_arrival'] / max_occupancy) * 100
df_data_h2.loc[:, 'occupancy_pct_at_departure'] = (df_data_h2.loc[:, 'occupancy_at_departure'] / max_occupancy) * 100

df_data_h2

In [ ]:
full_dataset = pd.concat([df_data_h1, df_data_h2], axis = 0)

full_dataset.to_feather('../../data/2.3_data_with_occupancies.feather', compression = 'lz4')

# Time Series Metrics and Analysis

In [ ]:
# # Assume df_data is the DataFrame name, and the data is already sorted by Arrival_Date in ascending order
# # and that Arrival_Date is in datetime format

# # Calculate the 7-day rolling average of occupancy_pct_at_arrival
# df_data_h1.loc[:, 'occupancy_pct_at_arrival_7d_avg'] = df_data_h1['occupancy_pct_at_arrival'].rolling(window=7, min_periods=1).mean()

# # Display the updated DataFrame to verify the calculation
# df_data_h1[['ArrivalDate', 'occupancy_pct_at_arrival', 'occupancy_pct_at_arrival_7d_avg']].head()


In [ ]:
# df_data_rollavg = df_data_h1[['ArrivalDate', 'occupancy_pct_at_arrival']].copy()
# df_data_rollavg

In [ ]:
# df_data_rollavg = df_data_rollavg.set_index(keys = 'ArrivalDate')
# df_data_rollavg

In [ ]:
# df_data_rollavg = df_data_rollavg.resample('D').mean()
# df_data_rollavg

In [ ]:
# df_data_rollavg.index

In [ ]:
# # Assume df_data is the DataFrame name, and the data is already sorted by Arrival_Date in ascending order
# # and that Arrival_Date is in datetime format

# # Calculate the 7-day rolling average of occupancy_pct_at_arrival
# df_data_rollavg.loc[:,'occupancy_pct_at_arrival_3d_avg'] = df_data_rollavg['occupancy_pct_at_arrival'].rolling(window=3, min_periods=1).mean()
# df_data_rollavg.loc[:,'occupancy_pct_at_arrival_7d_avg'] = df_data_rollavg['occupancy_pct_at_arrival'].rolling(window=7, min_periods=1).mean()
# df_data_rollavg.loc[:,'occupancy_pct_at_arrival_14d_avg'] = df_data_rollavg['occupancy_pct_at_arrival'].rolling(window=14, min_periods=1).mean()
# df_data_rollavg.loc[:,'occupancy_pct_at_arrival_28d_avg'] = df_data_rollavg['occupancy_pct_at_arrival'].rolling(window=28, min_periods=1).mean()
# # df_data_rollavg['occupancy_pct_at_arrival_90d_avg'] = df_data_rollavg['occupancy_pct_at_arrival'].rolling(window=90, min_periods=1).mean()

# # Display the updated DataFrame to verify the calculation
# df_data_rollavg.head()

In [ ]:
# px.line(df_data_rollavg)

In [ ]:
# rolling_avg_bookings = df_data_rollavg['occupancy_pct_at_arrival_7d_avg']

In [ ]:
# # Define high-demand threshold as the 75th percentile of the 7-day rolling average
# high_demand_threshold = rolling_avg_bookings.quantile(0.75).round(4)
# high_demand_threshold

In [ ]:
# # Identify days that are considered high demand
# high_demand_days = rolling_avg_bookings[rolling_avg_bookings > high_demand_threshold].index
# high_demand_days

In [ ]:
# # Initialize the indicator column with 0 (normal pricing)
# df_data['Dynamic_Pricing_Indicator'] = 0

# # For each booking, check if the booking date falls within a high-demand period
# # Assuming Booking_Date is already in datetime format and corresponds to the date the booking was made
# for booking_date in df_data['Booking_Date']:
#     if booking_date in high_demand_days:
#         df_data.loc[df_data['Booking_Date'] == booking_date, 'Dynamic_Pricing_Indicator'] = 1


In [ ]:
# df_data.loc[:,'Dynamic_Pricing_Indicator'].value_counts(dropna=0, normalize = 1, ascending=0)

In [ ]:
# df_data.head()